# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, load_psp("hgh/lda/Si-q4"))
atoms = [Si, Si]
positions =  [ones(3)/8, -ones(3)/8]

# We take very (very) crude parameters
model = model_DFT(lattice, atoms, positions; functionals=LDA())
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[1, 1, 1]);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, info0; maxiter)
    mixing_factor = .7
    x = x0
    info = info0
    for n = 1:maxiter
        fx, info = f(x, info)
        if info.converged || info.timedout
            break
        end
        x = x + mixing_factor * (fx - x)
    end
    (; fixpoint=x, info)
end;

Note that the fixpoint map `f` operates on an auxiliary variable `info` for
state bookkeeping. Early termination criteria are flagged from inside
the function `f` using boolean flags `info.converged` and `info.timedout`.
For control over these criteria, see the `is_converged` and `maxtime`
keyword arguments of `self_consistent_field`.

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (; λ, X, residual_norms=[], n_iter=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.
In the mixing interface `δF` is $(ρ_\text{out} - ρ_\text{in})$, i.e.
the difference in density between two subsequent SCF steps and the `mix`
function returns $δρ$, which is added to $ρ_\text{in}$ to yield $ρ_\text{next}$,
the density for the next SCF step.

In [4]:
struct MyMixing
    n_simple  # Number of iterations for simple mixing
end
MyMixing() = MyMixing(2)

function DFTK.mix_density(mixing::MyMixing, basis, δF; n_iter, kwargs...)
    if n_iter <= mixing.n_simple
        return δF  # Simple mixing -> Do not modify update at all
    else
        # Use the default KerkerMixing from DFTK
        DFTK.mix_density(KerkerMixing(), basis, δF; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-4,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.090073978406                   -0.39    0.0    1.57s
  2   -7.226157343674       -0.87       -0.65    0.0    1.26s
  3   -7.249782177445       -1.63       -1.17    0.0    130ms
  4   -7.250986294028       -2.92       -1.48    0.0   72.3ms
  5   -7.251258312426       -3.57       -1.78    0.0   74.2ms
  6   -7.251319808416       -4.21       -2.07    0.0    115ms
  7   -7.251334099764       -4.84       -2.35    0.0   72.5ms
  8   -7.251337569219       -5.46       -2.62    0.0   80.2ms
  9   -7.251338457710       -6.05       -2.88    0.0   93.1ms
 10   -7.251338698748       -6.62       -3.14    0.0   71.0ms
 11   -7.251338767946       -7.16       -3.39    0.0   80.1ms
 12   -7.251338788853       -7.68       -3.64    0.0   72.3ms
 13   -7.251338795449       -8.18       -3.88    0.0   79.0ms
 14   -7.251338797603       -8.67       -4.12    0.0   77.3ms


Note that the default convergence criterion is the difference in
density. When this gets below `tol`, the fixed-point solver terminates.
You can also customize this with the `is_converged` keyword argument to
`self_consistent_field`, as shown below.

## Customizing the convergence criterion
Here is an example of a defining a custom convergence criterion and specifying
it using the `is_converged` callback keyword to `self_consistent_field`.

In [6]:
function my_convergence_criterion(info)
    tol = 1e-10
    length(info.history_Etot) < 2 && return false
    ΔE = (info.history_Etot[end-1] - info.history_Etot[end])
    ΔE < tol
end

scfres2 = self_consistent_field(basis;
                                solver=my_fp_solver,
                                is_converged=my_convergence_criterion,
                                eigensolver=my_eig_solver,
                                mixing=MyMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.090073978406                   -0.39    0.0    186ms
  2   -7.226157343674       -0.87       -0.65    0.0    137ms
  3   -7.249782177445       -1.63       -1.17    0.0   71.2ms
  4   -7.250986294028       -2.92       -1.48    0.0    108ms
  5   -7.251258312426       -3.57       -1.78    0.0   72.5ms
  6   -7.251319808416       -4.21       -2.07    0.0   73.0ms
  7   -7.251334099764       -4.84       -2.35    0.0    101ms
  8   -7.251337569219       -5.46       -2.62    0.0   71.5ms
  9   -7.251338457710       -6.05       -2.88    0.0   76.9ms
 10   -7.251338698748       -6.62       -3.14    0.0   83.8ms
 11   -7.251338767946       -7.16       -3.39    0.0   72.6ms
 12   -7.251338788853       -7.68       -3.64    0.0   78.6ms
 13   -7.251338795449       -8.18       -3.88    0.0   96.8ms
 14   -7.251338797603       -8.67       -4.12    0.0   71.5ms
 15   -7.